# Library 📚

In [1]:
import os, time,csv,re, unicodedata, pandas as pd
from typing import List, Optional
from dotenv import load_dotenv

from urllib.parse import urlparse
import trafilatura, requests
from bs4 import BeautifulSoup

from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from textwrap import wrap

In [2]:
try:
    from langchain_mistralai import ChatMistralAI   # nouveau paquet
except ImportError:
    # fallback anciens imports si besoin
    from langchain_community.chat_models import ChatMistralAI

# Mini LLM Mistral

In [3]:
load_dotenv()
Mistral_key=os.getenv("Mistral_key")
LLM_MODEL='open-mixtral-8x7b'

llm = ChatMistralAI(
    model=LLM_MODEL,
    temperature=0,
    api_key=os.getenv("MISTRAL_API_KEY"),
    timeout=60,   
    max_retries=3,
)

In [4]:
CATEGORIES = ["Data Analyst","Data Scientist","Data Engineer","Analytics Engineer","Consultant Data","Autre"]
class ClassifOut(BaseModel):
    category: str = Field(description=f"Une des catégories: {', '.join(CATEGORIES)}")
    reasoning: Optional[str] = Field(default=None, description="Raison courte")

class SkillsOut(BaseModel):
    skills: List[str] = Field(description="Compétences techniques normalisées (Python, SQL, Airflow, dbt, Spark, Power BI…)")

classif_parser = PydanticOutputParser(pydantic_object=ClassifOut)
skills_parser = PydanticOutputParser(pydantic_object=SkillsOut)

## Prompts 🤖

In [5]:
classif_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Tu classes des intitulés de poste FR/EN en catégories fixes. "
     f"Catégories valides: {', '.join(CATEGORIES)}. "
     "Choisis UNE catégorie la plus appropriée. Réponds en JSON strict."),
    ("human",
     "Intitulé: {title}\n\nRappels: Data Analyst (analyste/BI/reporting), "
     "Data Scientist (ML/IA/recherche), Data Engineer (pipelines/ETL/Spark/warehouse), "
     "Analytics Engineer (dbt/semantic/BI engineering), Consultant Data (consultant spécialisé data), "
     "Autre sinon.\n\n{format_instructions}")
])

skills_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Tu reçois un texte d'offre (FR/EN). "
     "Liste UNIQUEMENT les compétences TECHNIQUES explicitement présentes dans le texte. "
     "N'invente rien. Normalise les noms (ex: 'scikit learn' → 'scikit-learn'). "
     "Réponds en JSON strict."),
    ("human",
     "Texte:\n\n{job_text}\n\n{format_instructions}")
])

## Chain 🔗

In [6]:
classif_chain = classif_prompt | llm | classif_parser
skills_chain = skills_prompt | llm | skills_parser

## LLM Calls ☎️

In [7]:
SKILL_LIST = [
    "python","sql","excel","tableau","power bi","looker","metabase",
    "airflow","dbt","spark","hadoop","databricks","kafka",
    "scikit-learn","pandas","numpy","pytorch","tensorflow","keras",
    "docker","kubernetes","git","mlflow","evidently","jenkins",
    "snowflake","redshift","bigquery","postgres","mongodb",
    "nlp","computer vision","xgboost","lightgbm",
    "aws","gcp","azure",
]
_skill_patterns = [re.compile(rf"\b{re.escape(s)}\b", re.I) for s in SKILL_LIST]
NORMALIZE = {"scikit learn": "scikit-learn", "powerbi": "power bi"}

def extract_skills(job_text: str) -> list:
    # 1) Règles d'abord (garantit un minimum)
    base = extract_skills_rules(job_text)

    # 2) Si le texte est court, on s’arrête là
    if not job_text or len(job_text) < 200:
        return base

    # 3) LLM (chunké) pour compléter
    chunks = wrap(job_text, 6000)
    llm_skills = set()
    for ch in chunks[:2]:  # max 2 chunks pour éviter timeouts/coûts
        try:
            out = skills_chain.invoke({
                "job_text": ch,
                "format_instructions": skills_parser.get_format_instructions()
            })
            for s in out.skills:
                s = s.strip()
                if s:
                    llm_skills.add(NORMALIZE.get(s.lower(), s))
        except Exception:
            continue

    # 4) Union + dédup + limite
    final = []
    seen = set()
    for s in base + sorted(llm_skills):
        k = s.lower()
        if k and k not in seen:
            seen.add(k)
            final.append(s)
    return final[:25]

def extract_skills_rules(text: str):
    """Extraction de compétences par simple recherche de mots-clés."""
    found = set()
    t = text or ""
    for s, pat in zip(SKILL_LIST, _skill_patterns):
        if pat.search(t):
            found.add(s)
    cleaned = []
    for s in found:
        cleaned.append(NORMALIZE.get(s.lower(), s))
    return sorted(set(cleaned))


In [8]:
def extract_skills_chunked(job_text: str) -> list:
    if not job_text:
        return []
    CHUNK = 6000
    chunks = wrap(job_text, CHUNK)
    all_skills = set(extract_skills_rules(job_text))  # base rules sur tout le texte
    for ch in chunks[:2]:  # limite à 2 chunks pour vitesse/coût
        try:
            out = skills_chain.invoke({
                "job_text": ch,
                "format_instructions": skills_parser.get_format_instructions()
            })
            for s in out.skills:
                if s.strip():
                    all_skills.add(s.strip())
        except Exception:
            continue
    return sorted(all_skills)[:25]

In [9]:
def clean_text(t: str) -> str:
    return " ".join((t or "").split())

def scrape_url(url: str, verbose: bool = True) -> str:
    if not isinstance(url, str) or not url.strip():
        return ""
    domain = urlparse(url).netloc.lower()

    # --- Essai A : requests -> trafilatura.extract (plus robuste, timeout côté requests)
    try:
        r = requests.get(url, timeout=15, headers={"User-Agent": "Mozilla/5.0"})
        if r.ok and r.text:
            extracted = trafilatura.extract(
                r.text,
                include_comments=False,
                include_tables=False,
                favor_recall=True,
            )
            if extracted:
                text = clean_text(extracted)
                if verbose:
                    print(f"  [scrape] {domain} via requests+trafilatura len={len(text)}")
                if len(text) > 120:
                    return text
    except Exception as e:
        if verbose:
            print(f"  [scrape] requests/trafilatura err: {e}")

    # --- Essai B : BeautifulSoup + sélecteurs spécifiques sites
    try:
        r = requests.get(url, timeout=15, headers={"User-Agent": "Mozilla/5.0"})
        if r.ok:
            soup = BeautifulSoup(r.text, "html.parser")

            # Sélecteurs spécifiques (adaptables) :
            if "welcometothejungle" in domain:
                main = soup.select_one("article, main, [data-testid='job-body']")
                if main: soup = main
            if "workable.com" in domain:
                main = soup.select_one("main, [data-ui='job-attributes'], [data-ui='job-description']")
                if main: soup = main
            if "smartrecruiters.com" in domain:
                main = soup.select_one("div#jobad, div#st-jo")
                if main: soup = main
            if "oraclecloud.com" in domain or "workday" in domain:
                main = soup.select_one("section[data-automation-id='jobPostingDescription'], section[data-automation-id='jobPostingHeader'], main, article")
                if main: soup = main

            # Nettoyage générique
            for tag in soup(["script","style","noscript","header","footer","nav"]):
                tag.decompose()

            text = clean_text(soup.get_text(" "))
            if verbose:
                print(f"  [scrape] {domain} via bs4 len={len(text)}")
            if len(text) > 120:
                return text
    except Exception as e:
        if verbose:
            print(f"  [scrape] bs4 err: {e}")

    if verbose:
        print(f"  [scrape] {domain} → texte insuffisant/indisponible")
    return ""




In [10]:
def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn").lower()

def classify_title(title: str):
    t = strip_accents(title)
    # heuristiques rapides
    if "analytics engineer" in t or ("analytics" in t and "engineer" in t) or "ingenieur analytics" in t:
        pre = "Analytics Engineer"
    elif "data engineer" in t or "ingenieur donnees" in t or "ingenieur data" in t:
        pre = "Data Engineer"
    elif "data scientist" in t or "scientifique des donnees" in t or "ml engineer" in t:
        pre = "Data Scientist"
    elif "data analyst" in t or "analyste" in t or "charge d etudes" in t or "bi analyst" in t or "business analyst" in t:
        pre = "Data Analyst"
    elif "consultant data" in t or "consultant ia" in t or "consultant bi" in t or "consultant analytics" in t:
        pre = "Consultant Data"
    else:
        pre = None

    if pre:
        return ClassifOut(category=pre, reasoning="heuristique")

    # sinon, LLM
    try:
        return classif_chain.invoke({
            "title": title or "",
            "format_instructions": classif_parser.get_format_instructions()
        })
    except Exception:
        return ClassifOut(category="Autre", reasoning=None)


# Pipeline 👨‍🔧

In [11]:
def run_pipeline(
    input_csv: str,
    output_csv: str = "data/suivi_candidatures_enrichi_mistral.csv",
    classify_all: bool = True,
    scrape_and_skills: bool = True,
    sleep_s: float = 0.6
):
    df = pd.read_csv(input_csv)

    if "Poste" not in df.columns:
        raise ValueError("Colonne 'Poste' requise.")

    for col in ["Catégorie Poste (LLM)", "Catégorie Raison (LLM)", "Offre (texte)", "Compétences (LLM)"]:
        if col not in df.columns:
            df[col] = None

    total_rows = len(df)
    print(f"[INFO] Démarrage pipeline sur {total_rows} lignes.")

    for i, row in df.iterrows():
        title = str(row.get("Poste", "") or "")
        link = str(row.get("Lien", "") or "")

        print(f"[INFO] Traitement ligne {i+1}/{total_rows} — Poste: {title[:50]}...")

        # 1) Classification
        if classify_all or pd.isna(row.get("Catégorie Poste (LLM)")):
            try:
                out = classify_title(title)
                df.at[i, "Catégorie Poste (LLM)"] = out.category
                df.at[i, "Catégorie Raison (LLM)"] = out.reasoning or ""
                print(f"  ↳ Catégorie: {out.category}")
            except Exception as e:
                df.at[i, "Catégorie Poste (LLM)"] = "Autre"
                df.at[i, "Catégorie Raison (LLM)"] = f"err:{type(e).__name__}"
                print(f"  [ERREUR] Classification: {e}")
            time.sleep(sleep_s)

        # 2) Scraping + extraction compétences
        if scrape_and_skills and link:
            print(f"  ↳ Scraping URL: {link}")
            text = scrape_url(link)
            df.at[i, "Offre (texte)"] = text

            if text:
                try:
                    skills = extract_skills(text)
                    df.at[i, "Compétences (LLM)"] = ", ".join(skills)
                    print(f"    ↳ Compétences trouvées: {len(skills)}")
                except Exception as e:
                    df.at[i, "Compétences (LLM)"] = ""
                    print(f"    [ERREUR] Extraction compétences: {e}")
            else:
                print(f"    [INFO] Aucun texte récupéré")
            time.sleep(sleep_s)

        # Sauvegarde intermédiaire toutes les 10 lignes
        if (i + 1) % 10 == 0:
            print(f"[INFO] Sauvegarde intermédiaire après {i+1} lignes...")
            os.makedirs(os.path.dirname(output_csv) or ".", exist_ok=True)
            for col in ["Compétences (LLM)", "Offre (texte)"]:
                if col in df.columns:
                    df[col] = df[col].astype(str)
            df.to_csv(
                output_csv,
                index=False,
                quoting=csv.QUOTE_MINIMAL,
                escapechar="\\",
                lineterminator="\n",
                encoding="utf-8",
            )

    # Sauvegarde finale
    print("[INFO] Sauvegarde finale du CSV...")
    os.makedirs(os.path.dirname(output_csv) or ".", exist_ok=True)
    for col in ["Compétences (LLM)", "Offre (texte)"]:
        if col in df.columns:
            df[col] = df[col].astype(str)
    df.to_csv(
        output_csv,
        index=False,
        quoting=csv.QUOTE_MINIMAL,
        escapechar="\\",
        lineterminator="\n",
        encoding="utf-8",
    )
    print(f"[OK] Fichier créé: {output_csv}")

# Start Button 🏌️‍♂️

In [12]:
if __name__ == "__main__":
    INPUT = "/teamspace/studios/this_studio/data/suivi_candidatures_V2.csv" 
    OUTPUT = "/teamspace/studios/this_studio/data/suivi_candidatures_enrichi_mistral.csv"
    run_pipeline(INPUT, OUTPUT)

[INFO] Démarrage pipeline sur 281 lignes.
[INFO] Traitement ligne 1/281 — Poste: Data Analyst Environnement...
  ↳ Catégorie: Data Analyst
  ↳ Scraping URL: https://www.welcometothejungle.com/fr/companies/inex-circular/jobs/data-analyst-environnement-cdi_paris_IC_y4Y4m9Z
  [scrape] www.welcometothejungle.com via requests+trafilatura len=4564
    ↳ Compétences trouvées: 5
[INFO] Traitement ligne 2/281 — Poste: Data Engineer Junior...
  ↳ Catégorie: Data Engineer
  ↳ Scraping URL: https://apply.workable.com/singulier/j/C258FD86BC/?utm_source=linkedin.com
  [scrape] apply.workable.com via bs4 len=32
  [scrape] apply.workable.com → texte insuffisant/indisponible
    [INFO] Aucun texte récupéré
[INFO] Traitement ligne 3/281 — Poste: Data Analyst...
  ↳ Catégorie: Data Analyst
  ↳ Scraping URL: https://www.welcometothejungle.com/fr/companies/arcep/jobs/data-analyst-f-h_paris-cedex-12
  [scrape] www.welcometothejungle.com via requests+trafilatura len=2957
    ↳ Compétences trouvées: 2
[INFO] 

# Final Result 🏆

In [13]:
mistral_df=pd.read_csv('data/suivi_candidatures_enrichi_mistral.csv')
mistral_df.head(50)

,Poste,Date de Soumission,État,Lien,Date de réponse,CV utilisé,Contact Email,Entreprise,Mois,Catégorie Poste,Catégorie Poste (LLM),Catégorie Raison (LLM),Offre (texte),Compétences (LLM)
0,Data Analyst Environnement,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV2,NaN,Inex Circular,7.0,Data Analyst,Data Analyst,heuristique,Cette offre n’est plus disponible. Résumé du p...,"airflow, excel, mongodb, pandas, python"
1,Data Engineer Junior,2025-07-31,Candidature envoyée,https://apply.workable.com/singulier/j/C258FD8...,NaN,CV2,NaN,Singulier,7.0,Data Engineer,Data Engineer,heuristique,NaN,NaN
2,Data Analyst,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV2,NaN,ARCEP,7.0,Data Analyst,Data Analyst,heuristique,Résumé du poste CDD / Temporaire(36 mois) Pari...,"excel, python"
3,Data scientist,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV2,NaN,Viginum,7.0,Data Scientist,Data Scientist,heuristique,Résumé du poste CDD / Temporaire(36 mois) Pari...,"computer vision, docker, git, nlp, pandas, python"
4,Data Scientist - Artifical Intelligence Projec...,2025-07-31,Refus,https://ekez.fa.em2.oraclecloud.com/hcmUI/Cand...,2025-08-01,CV2,NaN,Natixis,7.0,Data Scientist,Data Scientist,heuristique,NaN,NaN
5,"Data Quality Specialist, AI Tutor - Paris (Fix...",2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV2,NaN,Mistral AI,7.0,Autre,Autre,NaN,CDD / Temporaire Paris Télétravail total Salai...,NaN
6,CDI - Data Analyst - Petit Bateau !,2025-07-31,Candidature envoyée,https://www.linkedin.com/jobs/view/4274098673/...,NaN,CV2,NaN,Petit Bateau,7.0,Data Analyst,Data Analyst,heuristique,"We're signing you in Discover people, jobs, an...",NaN
7,Assistant Gestion Référentiels et Outils H/F,2025-07-31,Candidature envoyée,https://jobs.totalenergies.com/en_US/careers/A...,NaN,CV2,NaN,Total Energies,7.0,Autre,Autre,NaN,- Data Privacy1/4 - Personal Information2/4 - ...,NaN
8,Gestionnaire de données GED,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV3,NaN,RESAH,7.0,Autre,Autre,NaN,CDD / Temporaire(1 à 3 mois) Paris Télétravail...,excel
9,CDD - Analyste Surveillance des Engagements F/H,2025-07-31,Candidature envoyée,https://www.welcometothejungle.com/fr/companie...,NaN,CV3,NaN,La Banque Postale,7.0,Data Analyst,Data Analyst,heuristique,CDD / Temporaire(6 mois) Paris Télétravail fré...,"excel, power bi, sql"
